# Knapsack Problems
[Murwan Siddig](mailto:msiddig@clemson.edu)

----------------------------------------------------------------------

## (0-1) Knapsack Problem
Consider a set $N = \{1, 2, \dots, n\}$ with $n$ items, where:
* Each item $i \in N$ has a weight $w_i$ and a value $c_i$.
* Assumption: a fraction of an item has $0$ value. 
* Goal: maximize the total value of items in a knapsack which has a cpacity $b$.

### Decision variables: 
$
\Bigg\{
\begin{array}{cc}
& x_i = 1 ; & \text{If item} \ i \ \text{is included in the knapsack} \\
& x_i = 0 ; & \text{Otherwise} \\ 
\end{array}
\quad \forall \ i = 1, 2, \dots, n.
$

\begin{align*}
\max_{\text{s.t.}} \ & \sum_{i=1}^{n} c_i x_i   \\
& \sum_{i=1}^{n} w_i x_i \leq b & \\
& x_i \ \in \{0, 1\}, \ i=1, 2, \dots, n \\
\end{align*}

* Suppose your knapsack has **capacity of 8** and the **weights** and **values** of the items are given by:

|items |   1  |  2   |   3  |  4   |
|------|------|------|------|------|
|value | 30   |  60  |  70  |   50  |
|weight|  1   |  5   |  4   |   3  |      


In [1]:
#technical lines
using Random;
using Distributions;
using JuMP;
using Gurobi;
const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only


In [2]:
n= 4; #number of available items
b = 8; #capacity
c = [30, 60, 70, 50]; #values
w = [2, 5, 4, 3]; #weights

In [3]:
KS = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
#Define a decision variable for each item in the set N
@variable(KS, x[i=1:n], Bin); #Bin ==> is telling the solver x must be either 1 or 0.
@objective(KS, Max, sum(c[i]*x[i] for i=1:n));
@constraint(KS, sum(w[i]*x[i] for i=1:n)<= b);


#Solving the problem
optimize!(KS);
status = termination_status(KS);
obj_value = objective_value(KS);

#getting the value of the decision variables x
x_value = value.(x);
println("=======================================")
println("=======================================")
println("Status = ", status)
println("Optimal objective value = ", obj_value)
println("Optimal x value = ", x_value)

Status = OPTIMAL
Optimal objective value = 120.0
Optimal x value = [0.0, 0.0, 1.0, 1.0]


## A Dynamic Programming Formulation

In [4]:
function KS_DP(KnapSackCap,NumbItems)
    f = zeros(NumbItems+1,KnapSackCap+1); #this is the value of the knapsack equation while looking at the kth item with at capacity l 
    p = zeros(NumbItems+1,KnapSackCap+1); #this is the label of the item while looking at the kth item with at capacity l (0 means not taken, 1 otherwise) 
    for k=2:NumbItems+1
        for l=2:KnapSackCap+1
            f[k,l]=1e10
        end
    end
    
    for k=1:NumbItems
        for l=1:KnapSackCap
            if l+1 <= w[k] || f[k,l+1]>=c[k]+f[k,l+1-w[k]]
                f[k+1,l+1]=f[k,l+1];
                p[k+1,l+1]=0
            else
                f[k+1,l+1]=c[k]+f[k,l+1-w[k]];
                p[k+1,l+1]=1
            end
        end
    end
    return f, p
end         

KS_DP (generic function with 1 method)

In [5]:
KnapSackCap =  8;
NumbItems = 4;
KnapSack, labels = KS_DP(KnapSackCap,NumbItems);
items = [];
tempCap = KnapSackCap+1;
for i=NumbItems+1:-1:1
    if labels[i,tempCap]==1
        push!(items,i)
        tempCap -= w[i-1]
    end
end
items = items[end:-1:1].-1;
println("Items which will be included=" , items);

Items which will be included=[3, 4]


## Multiple items Knapsack Problem

**Now suppose there are multiple of each items and we can take as many as we want.**

### Decision variables: 
$
\begin{array}{cc}
& x_i : & \text{How many of item} \ i \ \text{should we include in the Knapsack} \\
\end{array}
\quad \forall \ i = 1, 2, \dots, n.
$

\begin{align*}
\max_{\text{s.t.}} \ & \sum_{i=1}^{n} c_i x_i   \\
& \sum_{i=1}^{n} w_i x_i \leq b & \\
& x_i \ \in \mathbb{Z}_{+}, \ i=1, 2, \dots, n \\
\end{align*}

In [6]:
b = 292; #capacity
c = [19, 55, 43, 33]; #values
w = [2, 5, 4, 3]; #weights

KS_integer = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
#Define a decision variable for each item in the set N
@variable(KS_integer, x[i=1:n]>=0, Int); #Int ==> is telling the solver x must be an integer value.
@objective(KS_integer, Max, sum(c[i]*x[i] for i=1:n));
@constraint(KS_integer, sum(w[i]*x[i] for i=1:n)<= b);


#Solving the problem
optimize!(KS_integer);
status = termination_status(KS_integer);
obj_value = objective_value(KS_integer);

#getting the value of the decision variables x
x_value = value.(x);
println("=======================================")
println("=======================================")
println("Status = ", status)
println("Optimal objective value = ", obj_value)
println("Optimal x value = ", x_value)

Status = OPTIMAL
Optimal objective value = 3212.0
Optimal x value = [0.0, 56.0, 0.0, 4.0]


## Dynamic Programming Formulation

In [7]:
values = c;
weights = w;
capacity =  b;

NumbItems = length(values)
new_items = [];
for k=1:NumbItems
    temp = fld(capacity,weights[k])
    push!(new_items,temp)
end

cc = zeros(convert(Int64,sum(new_items)))
aa = zeros(convert(Int64,sum(new_items)))

position = [];

count = 0
for k=1:NumbItems
    for n=1:new_items[k]
        templist = [];
        push!(templist,n)
        push!(templist,k)
        push!(position,templist)
        count +=1
        cc[count] = values[k]*n
        aa[count] = weights[k]*n
    end
end
    
NumbItems = convert(Int64,sum(new_items));
values = []
weights = []
for i=1:NumbItems
    tempc = convert(Int64, cc[i])
    push!(values,tempc)
    tempa = convert(Int64, aa[i])
    push!(weights,tempa)
end

In [8]:
function Knapsack(values, weights, capacity)
    len1 = convert(Int64,length(values)+1);
    println()
    len2 = convert(Int64,capacity+1);
    f = zeros(len1,len2);
    p = zeros(len1,len2); 
    for k=2:len1
        for l=2:len2
            f[k,l]=1e10
        end
    end
    
    for k=1:length(values)
        for l=1:capacity
            if l+1 <= weights[k] || f[k,l+1]>=values[k]+f[k,l+1-weights[k]]
                f[k+1,l+1]=f[k,l+1];
                p[k+1,l+1]=0
            else
                f[k+1,l+1]=values[k]+f[k,l+1-weights[k]];
                p[k+1,l+1]=1
            end
        end
    end
    return f, p
end

Knapsack (generic function with 1 method)

In [9]:
KnapSack, labels = Knapsack(values, weights, capacity);

println("Optimal value = " , KnapSack[length(values)+1,capacity+1])

items = [];
tempCap = capacity+1;
for i=length(values)+1:-1:1
    if labels[i,tempCap]==1
        push!(items,i)
        tempCap -= weights[i-1]
    end
end
items = items[end:-1:1].-1;
#println("Items which will be included=" , items)
println("Items which will be included:")
for i=1:length(items)
    println(position[items[i]][1], " items of ", position[items[i]][2])
end

#To double check our result
Optimal = sum(values[k] for k in items);


Optimal value = 3212.0
Items which will be included:
1 items of 2
2 items of 2
3 items of 2
4 items of 2
5 items of 2
6 items of 2
7 items of 2
8 items of 2
9 items of 2
11 items of 2
1 items of 4
3 items of 4
